In [1]:
!pip install datasets transformers==4.30.0

  Using cached transformers-4.30.0-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [49 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-312\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers
  creating build\lib.win-amd64-cpython-312\tokenizers\models
  copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\models
  creating build\lib.win-amd64-cpython-312\tokenizers\decoders
  copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\decoders
  creating build\lib.win-amd64-cpython-312\tokenizers\normalizers
  copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\normalizers
  creating build\lib.win-amd64-cpython-312\tokenizers\pre_tokenizers
  copying py_src\tokenizers\pre_tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\pre_tokenizers
  creating build\lib.win-amd64-c

In [3]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [11]:
class DataTransformation:
    def __init__(self):
        self.data_path = os.path.join("../artifacts", "data_ingestion", "wechatbot.csv")
        self.transformation_path = os.path.join("../artifacts","data_transformation")
        self.tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

    def load_dataframe(self):
        self.df = pd.read_csv(self.data_path)
        self.transformed_df = pd.DataFrame(columns=["input","output"])

    def save_csv(self, dataframe, location, filename):
        if not os.path.exists(location):
            os.makedirs(location, exist_ok=True)

        dataframe.to_csv(os.path.join(location, (filename + ".csv")), index=False)

    def direct_question(self):
        for index, row in self.df.iterrows():
            input_output_dict = {}
            list_of_topics = ["description", "session price", "lessons", "duration", "price"]
            
            for topic_index, topic in enumerate(list_of_topics):
                question = "what is the {flag1} of the course {flag2}".format(flag1 = topic, flag2 = row[0])        
                answer = "the {flag1} is: {flag2}".format(flag1 = topic, flag2 = row[topic_index + 1])
                input_output_dict["input"] = question
                input_output_dict["output"] = answer
                new_row = pd.DataFrame([input_output_dict])  # Create DataFrame from dict
                self.transformed_df = pd.concat([self.transformed_df, new_row], ignore_index=True)
        
                
        self.save_csv(dataframe=self.transformed_df, location=self.transformation_path, filename="transformed")

    def preprocessing(self, examples):
        model_inputs = self.tokenizer(
            examples['input_text'], 
            max_length=512, 
            truncation=True, 
            padding='max_length'  # Add padding to the max length
        )
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                examples['target_text'], 
                max_length=512, 
                truncation=True, 
                padding='max_length'  # Add padding to the max length
            )
    
        model_inputs['labels'] = labels['input_ids']
        return model_inputs

    def convert_to_features(self):
        train_data = self.transformed_df[['input', 'output']].to_dict(orient='records')
        dataset = Dataset.from_dict({'input_text': [item['input'] for item in train_data],
                             'target_text': [item['output'] for item in train_data]})
        
        train_test = dataset.train_test_split(test_size=0.1)  # 10% for evaluation
        train_dataset = train_test['train']
        eval_dataset = train_test['test']

        tokenized_dataset_train = train_dataset.map(self.preprocessing, batched=True)
        tokenized_dataset_eval = eval_dataset.map(self.preprocessing, batched=True)

        tokenized_dataset_train.save_to_disk(os.path.join(self.transformation_path, "train"))
        tokenized_dataset_eval.save_to_disk(os.path.join(self.transformation_path, "evaluate"))
        
    
    def initiate_data_transformation(self):
        self.load_dataframe()
        self.direct_question()
        self.convert_to_features()

In [13]:
data_transformation = DataTransformation()
data_transformation.initiate_data_transformation()

C:\Users\USER\AppData\Local\Temp\ipykernel_12548\885459174.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  question = "what is the {flag1} of the course {flag2}".format(flag1 = topic, flag2 = row[0])
C:\Users\USER\AppData\Local\Temp\ipykernel_12548\885459174.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = "the {flag1} is: {flag2}".format(flag1 = topic, flag2 = row[topic_index + 1])


Map:   0%|          | 0/301 [00:00<?, ? examples/s]

C:\Users\USER\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/301 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/34 [00:00<?, ? examples/s]